# 文本预处理
实现自然语言处理，首先要对待处理文本进行预处理。预处理的一般步骤有：读入文本、分词、建立字典、词序转换索引
## 读入文本

In [1]:
import collections
import re

def read_time_machine():
    with open('/home/kesci/input/timemachine7163/timemachine.txt', 'r') as f:
        # 按行处理，strip()移除开头结尾的字符，这里是空白；lower()将大写变为小写
        # re.sub()将用第三个属性的元素替换第二个属性的元素
        lines = [re.sub('[^a-z]+', ' ', line.strip().lower()) for line in f]
    return lines

lines = read_time_machine()  # 读入文本行数
print('# sentences {}'.format(len(lines)))
# print(lines)

# sentences 3221


## 分词
分词就是将句子划分成一堆词，将句子转换成词的序列

In [2]:
def tokenize(sentences, token='word'):
    """Split sentences into word or char tokens"""
    if token == 'word':  # 在单词级别做分词
        return [sentence.split(' ') for sentence in sentences]
    elif token == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR: unkown token type '+token)

tokens = tokenize(lines)
tokens[0:4]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells', ''], [''], [''], ['']]

## 建立字典
将文本转换成数字，以方便计算机处理；构建一个字典，将每个词映射到一个唯一的索引编号。

In [3]:
class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        # tokens存放所有的词,min_freq表示词频阈值，剔除词频很小的词
        # use_special_tokens表示是否使用特殊字符
        counter = count_corpus(tokens)  # 记录词出现的次数，返回键值对 
        self.token_freqs = list(counter.items())  # 将词提取出构成列表
        self.idx_to_token = []  # 存词的？？
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['pad', 'bos', 'eos', 'unk']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        self.idx_to_token += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):  # 类索引；词到索引的映射，给定词，返回索引
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):  # 索引到词的映射，给定索引，返回词
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

def count_corpus(sentences):
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)  # 返回一个字典，记录每个词的出现次数


In [4]:
# 用Time Machine作为语料构建字典
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('g', 6), ('wells', 7), ('i', 8), ('traveller', 9)]


## 将词转换为索引
利用字典将原文本中的句子从单词序列转换为索引序列

In [5]:
for i in range(8, 10):
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', '']
indices: [1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [20, 21, 22, 23, 24, 16, 25, 26, 27, 28, 29, 30]


## 用现有工具进行分词
前面的分词方式至少有以下几个缺点:
1.标点符号通常可以提供语义信息，但是我们的方法直接将其丢弃了
2.类似“shouldn't", "doesn't"这样的词会被错误地处理
3.类似"Mr.", "Dr."这样的词会被错误地处理
用两个工具spaCy和NLTK进行改善

In [6]:
text = "Mr. Chen doesn't agree with my suggestion."
# spacy
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
print([token.text for token in doc])
# NLTK
from nltk.tokenize import word_tokenize
from nltk import data
data.path.append('/home/kesci/input/nltk_data3784/nltk_data')
print(word_tokenize(text))

ModuleNotFoundError: No module named 'spacy'

# 语言模型
   自然语言处理要对文本进行处理，实现语音识别或机器翻译等文本的成型或转换，就不止需要把相应的读音转换成文本或将单个词翻译为另一语言的词，还需要考虑句子整体的连贯性，即要针对上下文来理解句子语义，以避免同音词的误读或一词多义导致的错译。对于我们人而言，我们需要学习语法，那么对于机器，我觉得，语言模型就可以看做是用来学习语法的工具。
我是这样理解的：给定一段长度为T的词的序列，语言模型将计算该序列的概率：P(w1, w2, …, wT)。假设我们要实现英译汉，给出一段英文文本序列，我们根据单词释义产生若干可能的汉语翻译文本，通过对每段翻译文本序列求概率并进行比较，就可以选出最优的翻译文本（概率最大的序列）。
   如何计算语言模型呢？因为序列中每个词是依次生成的，那么根据概率乘法公式，可得到：
![Image Name](https://cdn.kesci.com/upload/image/q5oqpynul4.PNG?imageView2/0/w/960/h/960)例如，一段含4个词的文本序列的概率为：![Image Name](https://cdn.kesci.com/upload/image/q5oqqf5m53.PNG?imageView2/0/w/960/h/960)
   那么，公式中的每个词的概率如何计算？
   我们假设用若干篇小说作为训练集来训练模型。这时，我们要求一段生成文本的概率，就要通过对训练集中这些词出现的次数进行统计。P(w1)就可以根据n(w1)/n来计算，n(A)代表A词在训练集出现的次数，n代表整个训练集的单词个数。P(w2|w1)就可以根据n(w1,w2)/n(w1)来计算，其中n(w1,w2)就是w1和w2作为整个单词出现的次数。这样，我们就可以计算出整个句子序列的概率，这就有点类似于我们人类经过学习可以通过语法语义对句子进行处理，而机器通过统计词频 “学习”根据语法语义处理句子。
## n元语法
   我们可以发现上面的概率计算理论上没问题，但实际中会随着句子长度的增加导致概率愈发地复杂。这时候，我们就要用到n元语法。n元语法是通过马尔可夫假设（不一定成立）简化语言模型的计算，即一个词的出现只与前面出现的n个词相关（n阶马尔可夫链）。基于n-1阶马尔可夫链，语言模型可表示成：
![Image Name](https://cdn.kesci.com/upload/image/q5oqs0ener.PNG?imageView2/0/w/960/h/960)
这个也叫n元语法(n-grams)。再用上面4个词的文本序列作为例子，3元语法的概率可表示为：
![Image Name](https://cdn.kesci.com/upload/image/q5oqsl6442.PNG?imageView2/0/w/960/h/960)

In [7]:
# 读取数据集
with open('/home/kesci/input/jaychou_lyrics4703/jaychou_lyrics.txt') as f:
    corpus_chars = f.read()
print(len(corpus_chars))
print(corpus_chars[: 40])
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[: 10000]

# 建立字符索引
idx_to_char = list(set(corpus_chars)) # 去重，得到索引到字符的映射，可用下表取出字符
char_to_idx = {char: i for i, char in enumerate(idx_to_char)} # 字符到索引的映射， 构建一个字典
vocab_size = len(char_to_idx)
print(vocab_size)

corpus_indices = [char_to_idx[char] for char in corpus_chars]  # 将每个字符转化为索引，得到一个索引的序列
sample = corpus_indices[: 20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

# 定义函数，后续使用
def load_data_jay_lyrics():
    with open('/home/kesci/input/jaychou_lyrics4703/jaychou_lyrics.txt') as f:
        corpus_chars = f.read()
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

63282
想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每
1027
chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [700, 718, 638, 805, 845, 104, 371, 700, 718, 733, 797, 551, 688, 632, 616, 533, 371, 700, 718, 733]


## 语言模型数据集
### 数据的采样
如果序列的长度为T，采样步数为n，那么一共有T-n个合法的样本，但是这些样本有大量的重合，我们通常采用更加高效的采样方式。我们有两种方式对数据进行采样，分别是随机采样和相邻采样。
### 随机采样
在随机采样中，每个样本是原始序列上任意截取的一段序列，相邻的两个随机小批量在原始序列上的位置不一定相毗邻。
![Image Name](https://cdn.kesci.com/upload/image/q5oqyr1656.PNG?imageView2/0/w/960/h/960)
如上图，随机采样根据时间步长将样本划分为若干等长的子序列，然后在每个子序列中选取批量值个数的数据进行批量处理。

In [8]:
# 随机采样
import torch
import random
def data_iter_random(corpus_indices, batch_size, num_steps, device=None):
    # 减1是因为对于长度为n的序列，X最多只有包含其中的前n - 1个字符
    num_examples = (len(corpus_indices) - 1) // num_steps  # 下取整，得到不重叠情况下的样本个数
    example_indices = [i * num_steps for i in range(num_examples)]  # 每个样本的第一个字符在corpus_indices中的下标
    random.shuffle(example_indices)

    def _data(i):
        # 返回从i开始的长为num_steps的序列
        return corpus_indices[i: i + num_steps]
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for i in range(0, num_examples, batch_size):
        # 每次选出batch_size个随机样本
        batch_indices = example_indices[i: i + batch_size]  # 当前batch的各个样本的首字符的下标
        X = [_data(j) for j in batch_indices]
        Y = [_data(j + 1) for j in batch_indices]
        yield torch.tensor(X, device=device), torch.tensor(Y, device=device)

my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [ 7,  8,  9, 10, 11, 12]]) 

X:  tensor([[18, 19, 20, 21, 22, 23],
        [12, 13, 14, 15, 16, 17]]) 
Y: tensor([[19, 20, 21, 22, 23, 24],
        [13, 14, 15, 16, 17, 18]]) 



### 相邻采样
在相邻采样中，相邻的两个随机小批量在原始序列上的位置相毗邻。
![Image Name](https://cdn.kesci.com/upload/image/q5or3vrnjz.PNG?imageView2/0/w/960/h/960)
如图，相邻采样将样本划分为等步长的子序列，然后在每个子序列中选取等批量值的数据组成一个批量。

In [9]:
# 相邻采样
def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size  # 保留下来的序列的长度
    corpus_indices = corpus_indices[: corpus_len]  # 仅保留前corpus_len个字符
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
        
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print('X: ', X, '\nY:', Y, '\n')

X:  tensor([[ 0,  1,  2,  3,  4,  5],
        [15, 16, 17, 18, 19, 20]]) 
Y: tensor([[ 1,  2,  3,  4,  5,  6],
        [16, 17, 18, 19, 20, 21]]) 

X:  tensor([[ 6,  7,  8,  9, 10, 11],
        [21, 22, 23, 24, 25, 26]]) 
Y: tensor([[ 7,  8,  9, 10, 11, 12],
        [22, 23, 24, 25, 26, 27]]) 

